In [1]:
from openai import OpenAI
import json

client = OpenAI()
# Define the number of entries to generate
dataset_size = 2  # Replace with your desired dataset size

# Initialize an empty list to store dataset entries
dataset = []

# Define dynamic actions and objects
object_context_template = [
    "Apple on the shelf (5m), red valve (2m)",
    "Box on the floor (3m), green button (4m)",
    "Chair in the corner (6m), yellow light switch (2m)"
]

available_actions_template = [
    ["NavigateTo", "Pick", "PressButton"],
    ["NavigateTo", "TurnLightOn", "MoveBox"],
    ["NavigateTo", "CloseValve", "PickUpChair"]
]

# Loop to generate dataset entries
for i in range(dataset_size):
    # Randomly select object context and available actions
    object_context = object_context_template[i % len(object_context_template)]
    available_actions = available_actions_template[i % len(available_actions_template)]
    
    # Step 1: Generate a human-like query
    query_prompt = f"""
You are simulating a human planning a task for a robot. 
Object Context: {object_context}
Available Actions: {', '.join(available_actions)}
Generate a query for the robot based on the given context and actions.
"""
    
    query_response = client.chat.completions.create(
        model="gpt-4o-mini",  # Update with your desired model
        messages=[
            {"role": "system", "content": "You are an assistant."},
            {"role": "user", "content": query_prompt}
        ],
        max_tokens=50,
        temperature=0.7
    )
    
    query = query_response.choices[0].message.content.strip()
    
    # Step 2: Generate the answer for the query
    answer_prompt = f"""
Object Context: {object_context}
Available Actions: {', '.join(available_actions)}
Query: {query}
Generate an action plan for the robot to execute this task. Include a behavior tree or state if the task is impossible.
"""
    
    answer_response = client.chat.completions.create(
        model="gpt-4",  # Update with your desired model
        messages=[
            {"role": "system", "content": "You are an assistant."},
            {"role": "user", "content": answer_prompt}
        ],
        max_tokens=150,
        temperature=0.7
    )
    
    answer = answer_response.choices[0].message.content.strip()
    
    # Example BT for this case
    if "impossible" in answer.lower():
        bt = "<?xml version=\"1.0\"?>\n<root main_tree_to_execute=\"BehaviorTree\">\n<EmptyTree/>\n</root>"
    else:
        bt = f"""<?xml version="1.0"?>
<root main_tree_to_execute="BehaviorTree">
    <BehaviorTree ID="GeneratedPlan">
        <Sequence name="TaskSequence">
            <Task description="{query}"/>
        </Sequence>
    </BehaviorTree>
</root>"""
    
    # Append the entry to the dataset
    dataset_entry = {
        "object_context": object_context,
        "available_actions": available_actions,
        "query": query,
        "answer": answer,
        "behavior_tree": bt
    }
    dataset.append(dataset_entry)

# Save the dataset to a JSON file
output_file = "synthetic_robot_action_dataset.json"
with open(output_file, "w") as file:
    json.dump(dataset, file, indent=4)

print(f"Dataset of {dataset_size} entries saved to {output_file}")

Dataset of 2 entries saved to synthetic_robot_action_dataset.json


In [2]:
import json
import pandas as pd

json_file = "captions_val2017.json"

with open(json_file, 'r') as f:
    coco_data = json.load(f)

# Convert to DataFrame - adjust these fields based on what you need from the COCO data
df = pd.DataFrame(coco_data['annotations'])
df = df[df['caption'].str.split().str.len() < 12]

df = df.drop(columns=['id', 'image_id'])

# convert to json
df.to_json('captions.json', orient='records', lines=True)

In [3]:
json_file = "objects.json"

with open(json_file, 'r') as f:
    coco_data = json.load(f)


df = pd.DataFrame(coco_data)


In [4]:
df.head()

,image_id,objects,image_url,merged_object_ids
0,1,"[{'synsets': ['tree.n.01'], 'h': 557, 'object_...",https://cs.stanford.edu/people/rak248/VG_100K_...,NaN
1,2,"[{'synsets': ['road.n.01'], 'h': 254, 'object_...",https://cs.stanford.edu/people/rak248/VG_100K_...,NaN
2,3,"[{'synsets': ['booth.n.02'], 'h': 389, 'object...",https://cs.stanford.edu/people/rak248/VG_100K_...,NaN
3,4,"[{'synsets': ['floor.n.01'], 'h': 168, 'object...",https://cs.stanford.edu/people/rak248/VG_100K_...,NaN
4,5,"[{'synsets': ['room.n.01'], 'h': 599, 'object_...",https://cs.stanford.edu/people/rak248/VG_100K_...,NaN
